# Drug Composition Analysis

## Problem Statement

### Part A:
Using the data from the OpenFDA API (documentation at https://open.fda.gov/apis/drug/label/) to determine the average number of ingredients (spl_product_data_elements) contained in AstraZeneca medicines per year.

1. Choose a method to gather the data
2. Transform the data as you see fit
3. Visualize and explore the results

The output should look similar to:

|year | drug_names | avg_number_of_ingredients|
|----|------|------|
|2018  | drugA,drugB  |  21|

### Part B:

Repeat the same analysis, calculate the average number of ingredients per year and per delivery route for all manufacturers.

The output should look similar to:

|year |  route |     avg_number_of_ingredients|
|---|----|-------|
|2018|   oral  |     123|
|2018|   injection|  213|
|2017 |  ...    |    ...|

### Part C:
Below are some questions to explore this toy problem further. 

Try creating some code if you still have time, or we can discuss some of these if you are invited for later stages of the interview:

* How would you code a model to predict the number of ingredients for next year? Note: Your predictions don't have to be good !
* Could you find the most common drug interactions for AstraZeneca medicines?
* How would you deploy your analysis/model as a tool that other users can use?



## Solution

## Import Dependent Libraries

Uncomment and Execute the following commands to install missing libraries

In [23]:
# !pip install gensim
# !pip install seaborn

In [114]:
import urllib.request as request
import re
import json 
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string

## 1. Retrieve data from OpenFDA API

In [2]:

class DataRetriever:
    #Constant variables declaration
    STARTDATE_YEAR, ENDDATE_YEAR = '0101', '1231'
    BASE_URL = "https://api.fda.gov/drug/label.json"
    QUERY_STRING = "?search=effective_time:[{0}+TO+{1}]&limit={2}"
    
    
    def __init__(self, year_start=2010, year_end=2020, limit=100):
        self.year_range = np.arange(year_start, year_end, 1)
        self.limit = limit
        
    def pull_data(self):
        ''' this method pulls the data from REST API of OpenFDA.
            @return
                consolidated dataframe object
        '''
        whole_data = pd.DataFrame()
        
        try:
            endpoints = self.construct_endpoint()

            for endpoint in endpoints:
                # Call the API using request method
                with request.urlopen(endpoint) as res:
                    res_data = res.read()
                    json_data = json.loads(res_data)

                    yearwise_df = pd.DataFrame(json_data['results'])
                    # Append the data row-wise
                    whole_data = whole_data.append(yearwise_df, ignore_index=True)
                    
        except Exception as e:
            raise Exception("Failed to collect data :{}".format(e))
            
        return whole_data
        
    
    def construct_endpoint(self):
        ''' Method to construct endpoint url with dynamic querystring'''
        endpoints = list()
        
        for y_val in self.year_range:
            start_date = str(y_val) + self.STARTDATE_YEAR
            end_date = str(y_val) + self.ENDDATE_YEAR
            # Construct query string and append to base url
            api_endpoint = self.BASE_URL + self.QUERY_STRING.format(start_date, end_date, self.limit)
            
            endpoints.append(api_endpoint)
            
        return endpoints

    
# Initialize object for retrieve data 
# the year range is set from 2015 to 2020 with 100 records
year_start = 2015
year_end = 2020
num_records = 100

api_conn = DataRetriever(year_start, year_end, num_records)

dataset = api_conn.pull_data()

In [3]:
# View first and last few records of the dataset
dataset.head(2).append(dataset.tail(2))

,effective_time,inactive_ingredient,keep_out_of_reach_of_children,purpose,warnings,questions,spl_product_data_elements,openfda,version,dosage_and_administration,...,patient_medication_information,precautions_table,disposal_and_waste_handling,drug_abuse_and_dependence_table,instructions_for_use,use_in_specific_populations_table,pediatric_use_table,instructions_for_use_table,inactive_ingredient_table,dosage_forms_and_strengths_table
0,20151102,[INACTIVE INGREDIENTS Sucrose],[Keep this and all medication out of reach of ...,"[USES Boils, Abscess, Otitis]",[WARNINGS This product is to be used for self-...,[QUESTIONS OR COMMENTS www.Rxhomeo.com | 1.888...,[SILICEA SILICEA SILICON DIOXIDE COLLOIDAL SIL...,{},1,"[DOSAGE Adults- Take 4 or 6 Pellets by mouth, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20150109,"[INGREDIENTS: TALC, POLYMETHYL METHACRYLATE, V...",[Keep out of reach of children If product is s...,[Purpose Sunscreen],[Warnings For external use only.],NaN,[CHANTECAILLE PROTECTION NATURELLE BRONZE SPF ...,{},4,[Directions Protection Naturelle SPF 46 PA+++ ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,20190401,NaN,NaN,NaN,[WARNINGS Risks from Concomitant Use with Opio...,NaN,[Clonazepam Clonazepam CLONAZEPAM CLONAZEPAM S...,{},10,[DOSAGE AND ADMINISTRATION Clonazepam is avail...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499,20190228,[Inactive Ingredient purified water],"[Keep out of reach of children If swallowed, g...",[Use For Preparation of Skin prior to an injec...,"[Warnings For External Use Only Flammable, Kee...",NaN,[Dukal Sterile Alcohol Prep Pad ISOPROPYL ALCO...,"{'product_ndc': ['65517-0002'], 'is_original_p...",4,[Other Information Store at Room Temperature 1...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Preparation

In [4]:
# Interested columns
col_interested = ['effective_time', 'inactive_ingredient', 'spl_product_data_elements', 'active_ingredient', 'purpose', 'indications_and_usage',  'openfda']

dataset[col_interested]

,effective_time,inactive_ingredient,spl_product_data_elements,active_ingredient,purpose,indications_and_usage,openfda
0,20151102,[INACTIVE INGREDIENTS Sucrose],[SILICEA SILICEA SILICON DIOXIDE COLLOIDAL SIL...,[ACTIVE INGREDIENT SILICEA HPUS 2X and higher],"[USES Boils, Abscess, Otitis]",[INDICATIONS Condition listed above or as dire...,{}
1,20150109,"[INGREDIENTS: TALC, POLYMETHYL METHACRYLATE, V...",[CHANTECAILLE PROTECTION NATURELLE BRONZE SPF ...,[BRONZE ACTIVE INGREDIENTS: TITANIUM DIOXIDE 2...,[Purpose Sunscreen],[Uses Multi-purpose mineral powder provides br...,{}
2,20151106,"[Inactive ingredients Ionized water, Carbamide]",[BAC-D Antibacterial Wound sanitizer Benzalkon...,[Active ingredient Benzalkonium chloride 0.13 %],[Purpose First aid antiseptic],[Uses First aid to help prevent the risk of ba...,{}
3,20150807,"[Inactive ingredients cedarleaf oil, fragrance...",[CVS Medicated Chest RubLavender scent Cough s...,[Active ingredients Camphor 4.8% Eucalyptus oi...,[Purpose Camphor - cough suppressant/topical a...,[Uses •temporarily relieves cough due to minor...,{}
4,20150828,[INACTIVE INGREDIENTS Sucrose/Lactose],[Iodoformum IODOFORM IODOFORM IODOFORM SUCROSE...,[ACTIVE INGREDIENTS IODOFORMUM],[USES To relieve the symptoms of cough.],[INDICATIONS Indications: IODOFORMUM Cough],"{'product_ndc': ['68428-959'], 'is_original_pa..."
...,...,...,...,...,...,...,...
495,20191023,NaN,[Furadantin Nitrofurantoin NITROFURANTOIN NITR...,NaN,NaN,[INDICATIONS & USAGE Furadantin is specificall...,{}
496,20191001,NaN,[Atorvastatin calcium Atorvastatin calcium ATO...,NaN,NaN,[1 INDICATIONS AND USAGE Therapy with lipid-al...,"{'product_ndc': ['71335-1336'], 'package_ndc':..."
497,20190909,"[Inactive ingredients hypromellose, lactose mo...",[cetirizine hydrochloride Cetirizine Hydrochlo...,[Active ingredient (in each tablet) Cetirizine...,[Purpose Antihistamine],[Uses temporarily relieves these symptoms due ...,"{'product_ndc': ['62011-0414'], 'package_ndc':..."
498,20190401,NaN,[Clonazepam Clonazepam CLONAZEPAM CLONAZEPAM S...,NaN,NaN,[INDICATIONS AND USAGE Seizure Disorders Clona...,{}


In [5]:
dataset[col_interested].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   effective_time             500 non-null    object
 1   inactive_ingredient        289 non-null    object
 2   spl_product_data_elements  500 non-null    object
 3   active_ingredient          282 non-null    object
 4   purpose                    279 non-null    object
 5   indications_and_usage      486 non-null    object
 6   openfda                    500 non-null    object
dtypes: object(7)
memory usage: 27.5+ KB


### 1. Cast appropriate data type


All the attributes of the dataset is in Object type. First we need to type cast the data fields to appropriate type for further processing. 

1. effective_time field to Datetime field
2. spl_product_data_elements, active_ingredient, inactive_ingredient attributes to string type

In [6]:
def cast_string(doc):
    if type(doc) != float:
        return str(doc[0])
    else:
        return np.nan

In [46]:
drug_data = dataset[col_interested]

# Type cast effective time attribute to Datetime
drug_data['effective_time'] = pd.to_datetime(drug_data['effective_time'])
drug_data['year'] = drug_data['effective_time'].dt.year

# String type fields
drug_data['spl_product_data_elements'] = drug_data['spl_product_data_elements'].apply(lambda x: str(x[0]))
drug_data['active_ingredient'] = drug_data['active_ingredient'].apply(cast_string)
drug_data['inactive_ingredient'] = drug_data['inactive_ingredient'].apply(cast_string)


### 2. Transform Drug and Ingredients data

In [47]:
print("Number of null or NAN values in product: {} ".format(drug_data['spl_product_data_elements'].isnull().sum()))
print("Number of null or NAN values in Active Ingredients: {} ".format(drug_data['active_ingredient'].isnull().sum()))
print("Number of null or NAN values in Inactive Ingredients: {} ".format(drug_data['inactive_ingredient'].isnull().sum()))

Number of null or NAN values in product: 0 
Number of null or NAN values in Active Ingredients: 218 
Number of null or NAN values in Inactive Ingredients: 211 


There are two types of Ingredients here,
1. Active Ingredients - Ingredients who have thearapautical value in the medicine
2. Inactive Ingredients - Which do not have therapautical values in the medicine but need for production of drug as a binding agent or vector for molecules. 

To find the Average Ingredients for the year, we need to identify the unique ingredient names for product. 

In [115]:
def clean_ingredient_text(doc):
    if type(doc) == str:
        # Remove punctuations from the document string
        clean_text = doc.translate(str.maketrans('', '', string.punctuation))
        
        pattern1 = re.compile(r'(inactive|active|ingredient|content|in each|tablet)[s]?', re.IGNORECASE)
        pattern2 = re.compile(r'\(.*\)', re.IGNORECASE)
        pattern3 = re.compile(r'\d.*(\%|\w)', re.IGNORECASE)
        pattern4 = re.compile(r'\d', re.IGNORECASE) # Clean all the numeric values
       
        
        doc_str = re.sub(pattern1, '', clean_text)
        doc_str = re.sub(pattern2, '', doc_str)
        doc_str = re.sub(pattern3, '', doc_str) # Making 'mg' as a marker to split the ingredient
        doc_str = re.sub(pattern4, '', doc_str)
        doc_str = re.sub(r'\s.?(mg)\s?', '-', doc_str)
        
        return doc_str
    else:
        return np.nan

def extract_route(doc):
    if not 'route' in doc.keys():
        np.nan
    else:
        return doc['route'][0]

def count_ingredient(doc):
    if type(doc) == str:
        word_list = set(word for word in doc.split('-') if len(word) > 1)
        return len(word_list)
    else:
        return 0
    
def count_inactive_ingredient(doc):
    if type(doc) == str:
        word_list = set(word for word in doc.split(' ') if len(word) > 1) # Return as a set to avoid duplicates
        return len(word_list)
    else:
        return 0

In [116]:
drug_data['active_ingredient'] = drug_data['active_ingredient'].apply(clean_ingredient_text)
drug_data['inactive_ingredient'] = drug_data['inactive_ingredient'].apply(clean_ingredient_text)
drug_data['route'] = drug_data['openfda'].apply(extract_route)

In [117]:
drug_data['route'] = drug_data['route'].ffill().bfill() 

In [118]:
drug_data['route'].values

array(['ORAL', 'ORAL', 'ORAL', 'ORAL', 'ORAL', 'ORAL', 'ORAL', 'ORAL',
       'ORAL', 'ORAL', 'ORAL', 'ORAL', 'TOPICAL', 'TOPICAL', 'TOPICAL',
       'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL',
       'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL',
       'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL',
       'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL',
       'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL',
       'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL',
       'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL',
       'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL',
       'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL',
       'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL',
       'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL',
       'TOPICAL', 'TOPICAL', 'TOPICAL', 'TOPICAL', 'TO

In [119]:
#drug_data['active_ingredient'].values

In [120]:
#drug_data['inactive_ingredient'][:50].values

In [121]:
drug_data['n_active_ingredient'] = drug_data['active_ingredient'].apply(count_ingredient)
drug_data['n_inactive_ingredient'] = drug_data['inactive_ingredient'].apply(count_inactive_ingredient)

In [122]:
#drug_data[['n_inactive_ingredient','n_ingredient']].values

In [123]:
# Adding active and inactive ingredient count to make total ingredients of the product
drug_data['n_total_ingredient'] = drug_data['n_active_ingredient'] + drug_data['n_inactive_ingredient']

In [127]:
drug_data.reset_index(drop=True, inplace=True)

In [128]:
drug_data.head()

,effective_time,inactive_ingredient,spl_product_data_elements,active_ingredient,purpose,indications_and_usage,openfda,year,n_active_ingredient,n_inactive_ingredient,n_total_ingredient,route
0,2015-11-02,Sucrose,SILICEA SILICEA SILICON DIOXIDE COLLOIDAL SILI...,SILICEA HPUS,"[USES Boils, Abscess, Otitis]",[INDICATIONS Condition listed above or as dire...,{},2015,1,1,2,ORAL
1,2015-01-09,TALC POLYMETHYL METHACRYLATE VINYL DIMETHICON...,CHANTECAILLE PROTECTION NATURELLE BRONZE SPF 4...,BRONZE TITANIUM DIOXIDE,[Purpose Sunscreen],[Uses Multi-purpose mineral powder provides br...,{},2015,1,21,22,ORAL
2,2015-11-06,Ionized water Carbamide,BAC-D Antibacterial Wound sanitizer Benzalkoni...,Benzalkonium chloride,[Purpose First aid antiseptic],[Uses First aid to help prevent the risk of ba...,{},2015,1,3,4,ORAL
3,2015-08-07,cedarleaf oil fragrance nutmeg oil thymol tu...,CVS Medicated Chest RubLavender scent Cough su...,Camphor,[Purpose Camphor - cough suppressant/topical a...,[Uses •temporarily relieves cough due to minor...,{},2015,1,8,9,ORAL
4,2015-08-28,SucroseLactose,Iodoformum IODOFORM IODOFORM IODOFORM SUCROSE ...,IODOFORMUM,[USES To relieve the symptoms of cough.],[INDICATIONS Indications: IODOFORMUM Cough],"{'product_ndc': ['68428-959'], 'is_original_pa...",2015,1,1,2,ORAL


In [113]:
cols = ['year', 'route']

drug_data['route'].unique()

array(['ORAL', 'TOPICAL', 'INTRAVENOUS', 'OPHTHALMIC', 'NASAL', 'DENTAL',
       'SUBLINGUAL', 'VAGINAL', 'RESPIRATORY (INHALATION)'], dtype=object)